In [18]:
import os
import csv
import json
import time
import datetime as dt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import ipywidgets as widgets
from dotenv import load_dotenv
from google.cloud import bigquery
load_dotenv()

sns.set(rc={'figure.figsize':(8,6)})
sns.set_style("darkgrid")

In [2]:
STRAVA_CLIENT_ID = os.getenv('STRAVA_CLIENT_ID')
STRAVA_CLIENT_SECRET = os.getenv('STRAVA_CLIENT_SECRET')
STRAVA_REFRESH_TOKEN = os.getenv('STRAVA_REFRESH_TOKEN')
STRAVA_TOKEN = json.loads(os.getenv('STRAVA_TOKEN').replace("\'", "\""))

# # For local running
!gcloud auth login
!gcloud config set project stavasnooper

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=lCP56Bu1g2weANRGw9UhfvI8fh9kti&access_type=offline&code_challenge=uaiHaaBvr_NAHdq1d6BTMTtefIC9utkzDuXGbFelaA4&code_challenge_method=S256


You are now logged in as [ty.elgin.andrews@gmail.com].
Your current project is [stavasnooper].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


In [17]:
bqclient = bigquery.Client(project='stavasnooper')
dataset_ref = client.dataset('prod_dashboard')
table_ref = dataset_ref.table('raw_strava_data')

max_request_date_query = """SELECT MAX(processed_timestamp) AS most_recent_processing
 FROM `stavasnooper.prod_dashboard.raw_strava_data` LIMIT 1000"""

try:
    latest_processing_date = bqclient.query(max_request_date_query).result().to_dataframe()
    latest_processing_date = latest_processing_date.most_recent_processing.iloc[0]
    print(latest_processing_date)
    if pd.isnull(latest_processing_date):
        print('get_latest_request_date: No processing timestamp in biquery table, using 2000-01-01 00:00:00')
        latest_processing_date = dt.datetime(2000, 1,1)
    print('get_latest_request_date: Last processed timestamp - {0}'.format(latest_processing_date))
    # start_date_dt = dt.datetime.strptime(latest_processing_date, "%Y-%m-%d %H:%M:%S")
    start_date_tuple = latest_processing_date.timetuple()
    start_date_unix = int(time.mktime(start_date_tuple))

except Exception as e:
    print(f"Issue with getting event log query results:\n {e}")

NaT
get_latest_request_date: No processing timestamp in biquery table, using 2000-01-01 00:00:00
get_latest_request_date: Last processed timestamp - 2000-01-01 00:00:00


In [11]:
latest_processing_date

NaT

In [7]:
from stravalib.client import Client
import requests

client = Client()
authorize_url = client.authorization_url(client_id=STRAVA_CLIENT_ID, redirect_uri='http://localhost:8282/authorized')
# Have the user click the authorization URL, a 'code' param will be added to the redirect_uri
# .....
# print(authorize_url)

r = requests.get('https://www.strava.com/oauth/authorize')

In [27]:
import requests
import json
import time
# Get the tokens from file to connect to Strava
# with open('strava_tokens.json') as json_file:
#     strava_tokens = json.load(json_file)
# If access_token has expired then 
# use the refresh_token to get the new access_token
if STRAVA_TOKEN['expires_at'] < time.time():
# Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': STRAVA_CLIENT_ID,
                                'client_secret': STRAVA_CLIENT_SECRET,
                                'grant_type': 'refresh_token',
                                'refresh_token': STRAVA_TOKEN['refresh_token']
                                }
                    )
# Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    # with open('strava_tokens.json', 'w') as outfile:
    #     json.dump(new_strava_tokens, outfile)
# Use new Strava tokens from now
    STRAVA_TOKEN = new_strava_tokens
# Open the new JSON file and print the file contents 
# to check it's worked properly
# with open('strava_tokens.json') as check:
#   data = json.load(check)
# print(data)
# print(json.dumps(data))


In [39]:
#Loop through all activities
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = STRAVA_TOKEN['access_token']

## Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame(
    columns = [
            "id",
            "name",
            "start_date_local",
            "type",
            "distance",
            "moving_time",
            "elapsed_time",
            "total_elevation_gain",
            "end_latlng",
            "external_id"
    ]
)
while page <= 2:
    
    # get page of activities from Strava
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
# if no results then exit loop
    if (not r):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        activities.loc[x + (page-1)*200,'id'] = r[x]['id']
        activities.loc[x + (page-1)*200,'name'] = r[x]['name']
        activities.loc[x + (page-1)*200,'start_date_local'] = r[x]['start_date_local']
        activities.loc[x + (page-1)*200,'type'] = r[x]['type']
        activities.loc[x + (page-1)*200,'distance'] = r[x]['distance']
        activities.loc[x + (page-1)*200,'moving_time'] = r[x]['moving_time']
        activities.loc[x + (page-1)*200,'elapsed_time'] = r[x]['elapsed_time']
        activities.loc[x + (page-1)*200,'total_elevation_gain'] = r[x]['total_elevation_gain']
        activities.loc[x + (page-1)*200,'end_latlng'] = r[x]['end_latlng']
        activities.loc[x + (page-1)*200,'external_id'] = r[x]['external_id']
# increment page
    print(f"Page {page:.0f} Data Received")
    page += 1
    
activities.to_csv('strava_activities.csv')

Page 1 Data Received
Page 2 Data Received


In [28]:
# store URL for activities endpoint
base_url = "https://www.strava.com/api/v3/"
endpoint = "athlete/activities"
url = base_url + endpoint
start_date_unix = get_latest_request_date()
# define headers and parameters for request
headers = {"Authorization": "Bearer {}".format(STRAVA_TOKEN['access_token'])}
params = {"after": start_date_unix,
        "per_page": 200,
        "page" : str(1) 
        }
# make GET request to Strava API
req = requests.get(url, headers = headers, params = params).json()

In [29]:
raw_df = pd.json_normalize(req)

In [30]:
raw_df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,external_id,...,map.summary_polyline,map.resource_state,average_watts,kilojoules,average_heartrate,max_heartrate,average_cadence,max_watts,weighted_average_watts,average_temp
0,2,2013-05-14,16868.2,2329,2593,148.8,Ride,NaN,54208005,347B4441-9E15-4BB8-BFED-35F7D09838A7,...,{xsvHjfwwT}@y@rL|MrOBbJ~KlJsAjAwO?gjB~AiXp[gI`...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2013-05-15,68916.8,8545,9061,959.1,Ride,NaN,54406970,270E68EC-1D74-4730-A83C-F6229AAEE3C5,...,_yauHznfwTdtDI\vnGMinGttDh@FjxVvFf_@NjMeBhKwx@...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2013-06-12,21636.5,3216,3373,271.9,Ride,NaN,59942311,AC05395A-E7A1-4C0C-A521-FD0EE28E01F6,...,_obvHduowTmDoFyGYUk_@fFwK`BkRSqYmIsKgZsL}F~@sF...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Afternoon Cyclocross ride,29943.2,6433,7788,148.5,Ride,NaN,80134614,1D88946F-3D2B-4C54-8F0B-C62332A0CBED,...,_p|fHbgyoVsIcLgOnAgOcLwLvBcLsSgc@vo@_DfOnKzOvV...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Afternoon Ride,51588.1,6992,7321,378.6,Ride,NaN,81585520,9C56699B-91CB-4828-907D-E642B8BA0830,...,k}zfHnmsoVvt@zr@ni@nAf^o`B~H{OcLkMjW?vQ_b@ja@c...,2,152.3,1065.0,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
def preprocess_raw_strava_data(raw_df, min_act_length=1200, max_act_dist=400, export=False):

    # processed_df = raw_df[(raw_df.elapsed_time > min_act_length) & (raw_df.distance < max_act_dist)].copy(deep=True)
    processed_df = raw_df.copy(deep=True)
    processed_df = processed_df.drop(processed_df[(processed_df.elapsed_time < min_act_length) | (processed_df.distance > max_act_dist*1000)].index)

    print(f'\t{len(raw_df[(raw_df.elapsed_time < min_act_length) | (raw_df.distance > max_act_dist*1000)])} Activities Under 20min in Length, Removed from Dataset')
    processed_df = processed_df.drop(columns=['athlete.resource_state', 'display_hide_heartrate_option', 'map.resource_state', 'map.summary_polyline', 'heartrate_opt_out',
                                'display_hide_heartrate_option', 'resource_state'])
    # processed_df = processed_df.convert_dtypes()
    processed_df[['distance_km']] = processed_df[['distance']]/1000
    processed_df[['start_date_local']] = pd.to_datetime(processed_df['start_date_local']) 
    processed_df[['start_date']] = pd.to_datetime(processed_df['start_date']) 
    processed_df['exer_start_time'] = pd.to_datetime(pd.to_datetime(processed_df['start_date_local']).dt.strftime('1990:01:01:%H:%M:%S'), format='1990:01:01:%H:%M:%S')
    processed_df['act_type_perc_time'] = (processed_df['moving_time']/sum(processed_df['moving_time']))
    processed_df['elapsed_time_hrs'] = (processed_df['elapsed_time']/3600)
    processed_df['moving_time_hrs'] = (processed_df['moving_time']/3600)
    # processed_df['distance_km'] = (processed_df['distance']/1000)

    processed_df.columns = processed_df.columns.str.replace("[.]", "_")


    return processed_df

In [46]:
processed_df = preprocess_raw_strava_data(raw_df)
processed_df.head()

# raw_df[(raw_df.elapsed_time < 1200) | (raw_df.distance > 400*1000)].index

# load_strava_data_into_bq(processed_df, 'processed_strava')

	3 Activities Under 20min in Length, Removed from Dataset


C:\Users\andrewt02\.conda\envs\tf2\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,external_id,upload_id,...,max_heartrate,average_cadence,max_watts,weighted_average_watts,average_temp,distance_km,exer_start_time,act_type_perc_time,elapsed_time_hrs,moving_time_hrs
0,2013-05-14,16868.2,2329,2593,148.8,Ride,NaN,54208005,347B4441-9E15-4BB8-BFED-35F7D09838A7,59752586.0,...,NaN,NaN,NaN,NaN,NaN,16.8682,1900-01-01 14:02:27,0.001472,0.720278,0.646944
1,2013-05-15,68916.8,8545,9061,959.1,Ride,NaN,54406970,270E68EC-1D74-4730-A83C-F6229AAEE3C5,59968531.0,...,NaN,NaN,NaN,NaN,NaN,68.9168,1900-01-01 13:45:34,0.005400,2.516944,2.373611
2,2013-06-12,21636.5,3216,3373,271.9,Ride,NaN,59942311,AC05395A-E7A1-4C0C-A521-FD0EE28E01F6,65850001.0,...,NaN,NaN,NaN,NaN,NaN,21.6365,1900-01-01 10:41:40,0.002033,0.936944,0.893333
3,Afternoon Cyclocross ride,29943.2,6433,7788,148.5,Ride,NaN,80134614,1D88946F-3D2B-4C54-8F0B-C62332A0CBED,87264324.0,...,NaN,NaN,NaN,NaN,NaN,29.9432,1900-01-01 15:18:37,0.004066,2.163333,1.786944
4,Afternoon Ride,51588.1,6992,7321,378.6,Ride,NaN,81585520,9C56699B-91CB-4828-907D-E642B8BA0830,88815514.0,...,NaN,NaN,NaN,NaN,NaN,51.5881,1900-01-01 13:31:47,0.004419,2.033611,1.942222


In [226]:
# storing number of activities
n_activities = len(processed_df)
# storing current date
current_date = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
# updating request log file
with open(os.path.join("..", 'data', 'raw', 'request_log.csv'), 'a', newline = '') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow([current_date, n_activities])

In [194]:


from google.oauth2 import service_account
from dotenv import load_dotenv
load_dotenv()

load_raw_strava_data_into_bq(req, 'raw_strava')

Running Load Job . 
JSON Load Job Status: RUNNING


In [217]:
from IPython.display import clear_output

def load_strava_data_into_bq(df, table_id):

    client = bigquery.Client(project='stavasnooper')
    # filename = '/path/to/file/in/nd-format.json'
    dataset_id = 'test'

    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job_config = bigquery.LoadJobConfig()
    # job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.autodetect = True

    job = client.load_table_from_dataframe(
            df,
            table_ref,
            location="us", # Must match the destination dataset location.
            job_config=job_config,
    )  # API request
    i=0
    while client.get_job(job.job_id, location='us').running():
        print("Running Load Job", "."*i, '\r',)
        i+=1
        time.sleep(1)

    print(f"Done Load Job.")

In [21]:
def get_latest_request_date():

    with open(os.path.join('..', 'data', 'raw', 'request_log.csv'), 'r') as f:
        # read file line-by-line     
        lines = f.read().splitlines()
        # store last line as a dictionary
        first_line = lines[0].split(',')
        last_line = lines[-1].split(',')
        last_line_dict = dict(list(zip(first_line, last_line)))
        # extract timestamp from last line
        start_date = last_line_dict['timestamp']
    # convert timestamp from ISO-8601 to UNIX format
    start_date_dt = dt.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    start_date_tuple = start_date_dt.timetuple()
    start_date_unix = int(time.mktime(start_date_tuple))

    return start_date_unix

In [49]:
import requests
from pandas.io.json import json_normalize
import json
import csv

# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()
    
df = json_normalize(r)
df.to_csv('strava_activities_all_fields.csv')

C:\Users\andrewt02\.conda\envs\tf2\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()
